<a href="https://colab.research.google.com/github/johnathan2012/Programming-iOS-Book-Examples/blob/master/GPT4Dev_ch10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10 設計 LINE AI 聊天機器人

## 10-3 OpenAI 變化圖像的功能

In [ ]:
!pip install rembg
!pip install gradio

from rembg import remove        # 匯入移除背景的函式
from PIL import Image           # 匯入 pillow 影像操作模組
import gradio as gr

def remove_bg(img, x_off, y_off, scale):
    x = int(x_off * img.width / 100)  # 計算橫向位移像數
    y = int(y_off * img.height / 100) # 計算縱向位移像數
    width = img.width - x             # 計算實際寬度
    height = img.height - y           # 計算實際高度
    size = min(width, height)         # 取寬高較短者
    img = img.crop((x, y, x + size, y + size)) # 切割正方形區域
    if scale < 100:
        size = int(size * scale / 100)# 依照指定比例縮放
        img = img.resize((size, size))
    img_nb = remove(img)              # 移除背景
    img.save('img.png')               # 儲存影像檔
    img_nb.save('img_nb.png')         # 儲存去背影像檔
    return (img, img_nb)

no_bg_if = gr.Interface(
    fn=remove_bg,
    inputs=[
        gr.Image(label='輸入影像', type='pil'),
        gr.Slider(label='橫向切割位移百分比'),
        gr.Slider(label='縱向切割位移百分比'),
        gr.Slider(label='縮放比例', value=100, step=5)],
    outputs=[gr.Gallery(label='處理後影像')]
)

no_bg_if.launch()

## 10-4 可控制變化內容的 create_edit 函式

In [ ]:
!pip install openai
from PIL import Image
from io import BytesIO
import requests
from google.colab import userdata

from openai import OpenAI
client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

In [ ]:
res = client.images.edit(
    image=open("image_nb.png", "rb"),
    prompt='冬日下大雪的路上',
    n=1,
    size="1024x1024"
)

In [ ]:
content = requests.get(res.data[0].url).content
img = Image.open(BytesIO(content))
from matplotlib import pyplot as plt
plt.axis("off")
plt.imshow(img)